In [8]:
%matplotlib inline
import matplotlib.pyplot as plt
import networkx as nx
from ipywidgets import interact
import pandas as pd
import itertools

from bokeh.io import show, output_notebook
from bokeh.plotting import figure

output_notebook()

Loading BokehJS ...

## Prepare the data

In [9]:
!ls

data1_back.csv	     data2-uniq.csv	      links	   related_pages.py
data1.csv	     fb-api.py		      links2	   scraped
data2.csv	     graph_viz		      links2-uniq  tmp.py
data2-uniq	     interactive_graphs.html  links-todo   visualize.ipynb
data2_uniq_back.csv  kaca-dp.py		      log


In [10]:
df1 = pd.read_csv('data/data1.csv',
                 sep=';',
                 header=None)

In [11]:
df1.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,cfact,HeartlandInstitute,MattKibbe,wattsupwiththat-133662869999306,Energy.Citizens,Reason.Magazine,justrealfacts,FoSClimateEd,The-Galileo-Movement-101728306584541,DavidAustinFrench,MercatorNet,CFACTCampus,marklevinshow,isthereglobalcooling,TheNewAmerican,hatingbreitbart,AmericanThinker,humaneventsmedia,IMConservative
1,iloveco2,inconvenientfacts.xyz,theGWPF,The-Galileo-Movement-101728306584541,thegrandsolarmini,wattsupwiththat-133662869999306,wwpagws,GunCultureWorld,ClimateHomeNews,LordMonckton,TheGeneralConsensus,iHeartClimateScientists,Armstrongecon,thepursuitofenergy,FoSClimateEd,TheLastAmericanVagabond,Michael-Roberts-blog-925340197491022,TheNewAmerican,R2ANM


In [12]:
df2 = pd.read_csv('data/data2-uniq.csv',
                 sep=';',
                 header=None)

In [13]:
df2.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,350.org,ExtinctionRebellion,WWF,CitizensClimateLobby,unionofconcernedscientists,greenpeaceusa,climatepower2020,EcoWatch,grist.org,nrdc.org,iucn.org,EnvDefenseFund,Earthjustice,Climate-Awareness-Network-325073881246240,auroraborealisobservatory,dontdrill,callforclimate,republicen,WorldWarZeroOrg
1,AboveClimateChange,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Create list of unique pages / nodes and page2id dict
uniq = []
for i, r in df1.iterrows():
    uniq.append(list(r))
    
for i, r in df2.iterrows():
    uniq.append(list(r))
    
uniq_pages = sorted([i for i in list(set(itertools.chain.from_iterable(uniq))) if isinstance(i, str)])
print(f"{len(uniq_pages)} nodes in total\n{uniq_pages}")

3455 nodes in total
['%D0%95%D1%80%D1%8A%D0%BD-%D0%92%D0%BE%D0%BD%D0%90%D0%BB%D1%8A%D0%BD-357344764330042', '%D0%98%D0%BD%D1%81%D1%82%D1%80%D1%83%D0%BC%D0%B5%D0%BD%D1%82%D0%B0%D0%BB%D0%B8%D1%82%D0%B5-%D1%81%D0%B0-%D1%82%D1%83%D0%BA-1880436278939206', '%E1%88%8E%E1%88%AC%E1%89%B5-%E1%8D%80%E1%8C%8B%E1%8B%A8-%E1%8C%88%E1%89%A5%E1%88%A8-%E1%88%98%E1%8B%B5%E1%88%85%E1%8A%95-686327608152311', '%E1%88%8E%E1%88%AC%E1%89%B5-%E1%8D%80%E1%8C%8B%E1%8B%AC-%E1%8C%88%E1%89%A5%E1%88%A8-%E1%88%98%E1%8B%B5%E1%88%85%E1%8A%95-271243023615527', '%E1%88%9B%E1%8A%95%E1%89%A0%E1%89%A5-%E1%88%99%E1%88%89-%E1%88%A0%E1%8B%8D-%E1%8B%AB%E1%8B%B0%E1%88%AD%E1%8C%8B%E1%88%8D-1136472886378018', '%E1%88%9B%E1%8A%95%E1%89%A0%E1%89%A5-%E1%88%99%E1%88%89-%E1%88%B0%E1%8B%8D-%E1%8B%AB%E1%8B%B0%E1%88%AD%E1%8C%8B%E1%88%8D-481659928701268', '%E1%88%B5%E1%88%88-%E1%8C%A4%E1%8A%93%E1%8B%8E-%E1%88%9D%E1%8A%95-%E1%8B%AB%E1%8B%8D%E1%89%83%E1%88%89-General-knowledge-910585735677691', '%E1%8A%A0%E1%88%9B%E1%88%AB-Times-1045064544884

In [15]:
# Create list of init pages and related_1 pages
initial = list(df1[0])
related_1 = list(df2[0])

In [16]:
page2id = {name: i for i, name in enumerate(uniq_pages)}
page2id

{'%D0%95%D1%80%D1%8A%D0%BD-%D0%92%D0%BE%D0%BD%D0%90%D0%BB%D1%8A%D0%BD-357344764330042': 0,
 '%D0%98%D0%BD%D1%81%D1%82%D1%80%D1%83%D0%BC%D0%B5%D0%BD%D1%82%D0%B0%D0%BB%D0%B8%D1%82%D0%B5-%D1%81%D0%B0-%D1%82%D1%83%D0%BA-1880436278939206': 1,
 '%E1%88%8E%E1%88%AC%E1%89%B5-%E1%8D%80%E1%8C%8B%E1%8B%A8-%E1%8C%88%E1%89%A5%E1%88%A8-%E1%88%98%E1%8B%B5%E1%88%85%E1%8A%95-686327608152311': 2,
 '%E1%88%8E%E1%88%AC%E1%89%B5-%E1%8D%80%E1%8C%8B%E1%8B%AC-%E1%8C%88%E1%89%A5%E1%88%A8-%E1%88%98%E1%8B%B5%E1%88%85%E1%8A%95-271243023615527': 3,
 '%E1%88%9B%E1%8A%95%E1%89%A0%E1%89%A5-%E1%88%99%E1%88%89-%E1%88%A0%E1%8B%8D-%E1%8B%AB%E1%8B%B0%E1%88%AD%E1%8C%8B%E1%88%8D-1136472886378018': 4,
 '%E1%88%9B%E1%8A%95%E1%89%A0%E1%89%A5-%E1%88%99%E1%88%89-%E1%88%B0%E1%8B%8D-%E1%8B%AB%E1%8B%B0%E1%88%AD%E1%8C%8B%E1%88%8D-481659928701268': 5,
 '%E1%88%B5%E1%88%88-%E1%8C%A4%E1%8A%93%E1%8B%8E-%E1%88%9D%E1%8A%95-%E1%8B%AB%E1%8B%8D%E1%89%83%E1%88%89-General-knowledge-910585735677691': 6,
 '%E1%8A%A0%E1%88%9B%E1%88%AB-Times-10450

## Create the graph

In [17]:
# directed =True: creates directed graph; undirected otherwise
# only_first =True: creates graph with only initial pages and their related pages (300 nodes)
#                   otherwise related of related pages are included as well (3500 nodes)

# !!!
# Inside function used variables calculated before -- not very smart!!!
def prepare_graph(directed=True, only_first=True):
    if directed:
        G = nx.DiGraph()
    else:
        G = nx.Graph()
    
    # Add nodes
    for page in uniq_pages:
        if page:
            if page in initial:
                G.add_node(page2id[page], name=page, layer='initial')
            elif page in related_1:
                G.add_node(page2id[page], name=page, layer='related_1')
            elif not only_first:
                G.add_node(page2id[page], name=page, layer='related_2')

    # Add edges
    for i, r in df1.iterrows():
        for ii, page in enumerate(r):
            if r[0] != page and isinstance(page, str):
                if r[0] not in page2id:
                    print(f"{r[0]} (init-like) missing in the page2id")
                if page not in page2id:
                    print(f"{page} (related) missing in the page2id")
                if r[0] in page2id and page in page2id:
                    G.add_edge(page2id[r[0]], page2id[page]) 
    
    if not only_first:
        for i, r in df2.iterrows():
            for ii, page in enumerate(r):
                if r[0] != page and isinstance(page, str):
                    if r[0] not in page2id:
                        print(f"{r[0]} (init-like) missing in the page2id")
                    if page not in page2id:
                        print(f"{page} (related) missing in the page2id")
                    if r[0] in page2id and page in page2id:
                        G.add_edge(page2id[r[0]], page2id[page])
    return G

G = prepare_graph()

In [18]:
print(f"Nodes: {len(G.nodes)}\nEdges: {len(G.edges)}")

Nodes: 287
Edges: 339


## Bokeh Visualizations

In [19]:
from bokeh.models import Range1d, Plot, HoverTool, BoxZoomTool, ResetTool, SaveTool, MultiLine, EdgesAndLinkedNodes
from bokeh.models import WheelZoomTool, PanTool
from bokeh.plotting import from_networkx
from bokeh.transform import factor_cmap
from bokeh.io import output_file
from bokeh.palettes import d3

# We could use figure here but don't want all the axes and titles
plot = Plot(x_range=Range1d(-2, 2),
            y_range=Range1d(-2, 2),
            plot_width=800,
            plot_height=800)
plot.add_tools(HoverTool(tooltips=[("name", "@name"), ("layer", "@layer")]),
               BoxZoomTool(),
               ResetTool(),
               SaveTool(),
               WheelZoomTool(),
               PanTool())
plot.title.text = "Related FB Pages"

# Create a Bokeh graph from the NetworkX input using nx.spring_layout
graph = from_networkx(G,
                      nx.spring_layout,
                      scale=1.8,
                      center=(0,0))


graph.node_renderer.data_source.data['colors'] = [n['layer'] if 'layer' in n else 'related_1' for i, n in G.nodes(data=True)]
plot.renderers.append(graph)

# Set some of the default node glyph (Circle) properties
graph.node_renderer.glyph.update(size=10, fill_color=factor_cmap('colors',
                                                                 factors=['initial', 'related_1', 'related_2'],
                                                                 palette=d3['Category10'][3]))


# Set some edge properties too
graph.edge_renderer.glyph = MultiLine(line_color='grey', line_alpha=0.5)
graph.edge_renderer.hover_glyph = MultiLine(line_color='green', line_width=2)
# graph.inspection_policy = EdgesAndLinkedNodes()

output_notebook()
show(plot)

# Save the plot
# output_file("interactive_graphs.html")

Loading BokehJS ...